In [1]:
# to import .py module
import sys
import os
ROOT_DIR = os.path.abspath(os.curdir)[:-11]
sys.path.append(os.path.abspath(ROOT_DIR))

In [2]:
import codecs
import json
import Preprocess
from Parameters import *
import numpy as np
import keras
from keras.models import Sequential, Model, load_model, save_model
from keras.layers import Embedding, Input, Dense, Reshape, Concatenate, Flatten, Bidirectional, LSTM, Activation, Dropout
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras.regularizers import l2
from time import time
import multiprocessing as mp
import sys
import math
import tensorflow as tf
import timeit
import CRF

### Bert model

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import json

def get_model(model_url, max_seq_length):
  labse_layer = hub.KerasLayer(model_url, trainable=True)

  # Define input.
  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                         name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                     name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="segment_ids")

  # LaBSE layer.
  pooled_output,  _ = labse_layer([input_word_ids, input_mask, segment_ids])

  # The embedding is l2 normalized.
  pooled_output = tf.keras.layers.Lambda(
      lambda x: tf.nn.l2_normalize(x, axis=1))(pooled_output)

  # Define model.
  return tf.keras.Model(
        inputs=[input_word_ids, input_mask, segment_ids],
        outputs=pooled_output), labse_layer

max_seq_length = 64
labse_model, labse_layer = get_model(
    model_url="https://tfhub.dev/google/LaBSE/1", max_seq_length=max_seq_length)

In [ ]:
import bert

vocab_file = labse_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = labse_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

def create_input(input_strings, tokenizer, max_seq_length):

  input_ids_all, input_mask_all, segment_ids_all = [], [], []
  for input_string in input_strings:
    # Tokenize input.
    input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    sequence_length = min(len(input_ids), max_seq_length)

    # Padding or truncation.
    if len(input_ids) >= max_seq_length:
      input_ids = input_ids[:max_seq_length]
    else:
      input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

    input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

    input_ids_all.append(input_ids)
    input_mask_all.append(input_mask)
    segment_ids_all.append([0] * max_seq_length)

  return np.array(input_ids_all), np.array(input_mask_all), np.array(segment_ids_all)

def encode(input_text):
  input_ids, input_mask, segment_ids = create_input(
    input_text, tokenizer, max_seq_length)
  return labse_model([input_ids, input_mask, segment_ids])

### Model

#### Load data and define necessary parameters, functions

In [ ]:
import json
with open(TRAIN_DATA_FILE, encoding='utf8') as f:
  trainset = json.load(f)

In [ ]:
with open(file=NORM_ADDS_FILE, mode='r', encoding='utf-8') as f:
  NORM_ADDS = json.load(fp=f)

with open(file=ID2id_FILE, mode='r', encoding='utf-8') as f:
  ID2id = json.load(fp=f)

with open(file=id2ID_FILE, mode='r', encoding='utf-8') as f:
  id2ID = json.load(fp=f)

with open(file=id2norm_add_FILE, mode='r', encoding='utf-8') as f:
  id2norm_add= json.load(fp=f)

In [ ]:
norm_embeddings = np.load(NORM_EMBEDDING_FILE, allow_pickle=True)
NT_norm_embeddings = np.load(NT_NORM_EMBEDDING_FILE, allow_pickle=True)

In [ ]:
entities2index = {'street': 0, 'ward': 1, 'district': 2, 'city': 3}

In [ ]:
def create_type_add_vector(noisy_add):
  entities = CRF.detect_entity(noisy_add)
  type_add_vector = np.zeros((1,4))
  for entity in entities:
    if entity == 'name':
      pass
    else:
      index = entities2index[entity]
      type_add_vector[0, index] = 1
  return type_add_vector

In [ ]:
# for a sample in trainset, get id of norm_add coresponding to noisy_add of this sample
def get_norm_id(sample):
  return list(sample['std_add'].keys())[0]

In [ ]:
# norm_embeddings.shape
dim = 772
num_of_norm = 34481
num_negative = 1023
SNN_layers = [512, 256, 128]
MLP_layers = [64, 32, 16, 8]

#### Generator

In [ ]:
def concat(v,type_add_vector):
  return np.concatenate((v, type_add_vector), axis=1)

In [ ]:
def generator(trainset, num_negative, id2norm_add, dim):
  # for i, sample in enumerate(trainset['data'][:100]):
  for i, sample in enumerate(trainset['data'][:32932]):
    batch_size = num_negative + 1
    batch_indexes = np.array([i for i in range(batch_size)])
    noisy_adds, norm_adds, labels = [], [], []
    # noisy add có thể hoa hoặc thường, đưa về chữ thường hết và loại bỏ đi punctuation
    noisy_add = preprocess.remove_punctuation(CRF.get_better_add(sample['noisy_add']).lower())

    id = get_norm_id(sample)
    ID_ = int(id2ID[id])

    type_add_vector = create_type_add_vector(sample['noisy_add'])
    noisy_add_vector = concat(np.array(encode([noisy_add])), type_add_vector)

    # kiểm tra xem noisy_add truyền vào là thường hay hoa
    if noisy_add != preprocess.remove_tone_of_text(noisy_add):
      noisy_adds.append(noisy_add_vector)
      norm_adds.append(norm_embeddings[ID_, :].reshape(1, -1))
      labels.append(1)

      # nagative sample
      for t in range(num_negative):
        j = str(np.random.randint(num_of_norm))
        while (j not in id2norm_add) or (j == id):
          j = str(np.random.randint(num_of_norm))

        ID = int(id2ID[j])
        noisy_adds.append(noisy_add_vector)
        norm_adds.append(norm_embeddings[ID, :].reshape(1, -1))
        labels.append(0)

    else:
      noisy_adds.append(noisy_add_vector)
      norm_adds.append(NT_norm_embeddings[ID_, :].reshape(1, -1))
      labels.append(1)

      # negative sample
      for t in range(num_negative):
        j = str(np.random.randint(num_of_norm))
        while (j not in id2norm_add) or (j == id):
          j = str(np.random.randint(num_of_norm))

        ID = int(id2ID[j])
        noisy_adds.append(noisy_add_vector)
        norm_adds.append(NT_norm_embeddings[ID, :].reshape(1, -1))
        labels.append(0)

    np.random.shuffle(batch_indexes)
    yield [np.array(noisy_adds).reshape(batch_size, dim)[batch_indexes], np.array(norm_adds).reshape(batch_size, dim)[batch_indexes]], np.array(labels)[batch_indexes]

#### Model

In [ ]:
def get_SNN_model(dim,
                  SNN_layers,
                  MLP_layers, 
                  type_ = None):
    num_SNN_layers = len(SNN_layers)

    noisy_input         = Input(shape=(dim,), dtype=np.float32, name='noisy_input')
    norm_input          = Input(shape=(dim,), dtype=np.float32, name='norm_input')

    if type_ == 'add':
      noisy_latent = noisy_input
      norm_latent = norm_input
      # SNN layer
      for idx in range(num_SNN_layers):
        Dense_layer = Dense(units=SNN_layers[idx]) 
        noisy_latent = Dense_layer(noisy_latent)
        norm_latent = Dense_layer(norm_latent)

        Activation_layer = keras.layers.LeakyReLU(alpha=0.1)
        noisy_latent = Activation_layer(noisy_latent)
        norm_latent = Activation_layer(norm_latent)

      predict_vector = keras.layers.Concatenate()([noisy_latent, norm_latent]) 

      # MLP layers
      for idx in range(len(MLP_layers)):
        Dense_layer = Dense(units=MLP_layers[idx]) 
        predict_vector = Dense_layer(predict_vector)

        Activation_layer = keras.layers.LeakyReLU(alpha=0.1)
        predict_vector = Activation_layer(predict_vector)

      # Add a dense layer with a sigmoid unit to generate the similarity score
      prediction = Dense(1,activation='sigmoid')(predict_vector)
      
      # Connect the inputs with the outputs
      siamese_net = keras.models.Model(inputs=[noisy_input,norm_input],outputs=prediction)
      
      # return the model
      return siamese_net
    elif type_ == 'cosine':
      noisy_latent = noisy_input
      norm_latent = norm_input
      # SNN layer
      for idx in range(num_SNN_layers):
        Dense_layer = Dense(units=SNN_layers[idx]) 
        noisy_latent = Dense_layer(noisy_latent)
        norm_latent = Dense_layer(norm_latent)

        Activation_layer = keras.layers.LeakyReLU(alpha=0.1)
        noisy_latent = Activation_layer(noisy_latent)
        norm_latent = Activation_layer(norm_latent)

      L1_layer = keras.layers.Lambda(lambda tensors:-keras.losses.cosine_similarity(tensors[0],tensors[1], axis=-1))
      prediction = L1_layer([noisy_latent, norm_latent])
      siamese_net = keras.models.Model(inputs=[noisy_input,norm_input],outputs=prediction)
      return siamese_net
keras.backend.clear_session()

#### Initialize model

In [ ]:
SNN_model = get_SNN_model(dim, SNN_layers, MLP_layers, type_='add')
SNN_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
noisy_input (InputLayer)        [(None, 772)]        0                                            
__________________________________________________________________________________________________
norm_input (InputLayer)         [(None, 772)]        0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 512)          395776      noisy_input[0][0]                
                                                                 norm_input[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_3 (LeakyReLU)       (None, 512)          0           dense_3[0][0]              

In [ ]:
tf.keras.utils.plot_model(SNN_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
SNN_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=keras.losses.binary_crossentropy)

### train

In [ ]:
losses = []
start = timeit.default_timer()
for i in range(20):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1967s 60ms/step - loss: 0.0118
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_1_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_1_epoches.snn/assets


32932/32932 [==============================] - 1977s 60ms/step - loss: 0.0025
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_2_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_2_epoches.snn/assets


   1562/Unknown - 98s 63ms/step - loss: 0.0020Buffered data was truncated after reaching the output size limit.

In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_2_epoches.snn')
start = timeit.default_timer()
for i in range(2, 20):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 2063s 63ms/step - loss: 0.0019
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_3_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_3_epoches.snn/assets


32932/32932 [==============================] - 2059s 63ms/step - loss: 0.0015
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_4_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_4_epoches.snn/assets


32932/32932 [==============================] - 2051s 62ms/step - loss: 0.0013
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_5_epoches.snn/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_11_epoches.snn/assets


32932/32932 [==============================] - 2067s 63ms/step - loss: 6.2899e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_12_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_12_epoches.snn/assets


   7421/Unknown - 475s 64ms/step - loss: 5.8777e-04

KeyboardInterrupt: ignored

In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_12_epoches.snn')
start = timeit.default_timer()
for i in range(12, 32):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1245s 38ms/step - loss: 5.7877e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_13_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_13_epoches.snn/assets


32932/32932 [==============================] - 1228s 37ms/step - loss: 5.4117e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_14_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_14_epoches.snn/assets


32932/32932 [==============================] - 1256s 38ms/step - loss: 4.9959e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_15_epoches.snn/assets


...

32932/32932 [==============================] - 1275s 39ms/step - loss: 2.3875e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_31_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_31_epoches.snn/assets


32932/32932 [==============================] - 1243s 38ms/step - loss: 2.3207e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_32_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_32_epoches.snn/assets


Time:  25168.971761365003
[0.0005787727423012257, 0.0005411722813732922, 0.0004995918716304004, 0.0004701571015175432, 0.00044737468124367297, 0.00042253921856172383, 0.00039958374691195786, 0.0003795509983319789, 0.00035464970278553665, 0.0003388154727872461, 0.00032348529202863574, 0.0003201212966814637, 0.00030312800663523376, 0.0002814946055877954, 0.00027238117763772607, 0.0002715411246754229, 0.00025319549604319036, 0.0002483712451066822, 0.00023875143961049616, 0.00023206508194562048]


In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_32_epoches.snn')
start = timeit.default_timer()
for i in range(32, 50):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1312s 40ms/step - loss: 2.1889e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_33_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_33_epoches.snn/assets


32932/32932 [==============================] - 1257s 38ms/step - loss: 2.1779e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_34_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_34_epoches.snn/assets


32932/32932 [==============================] - 1266s 38ms/step - loss: 2.0645e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_35_epoches.snn/assets


...

32932/32932 [==============================] - 1247s 38ms/step - loss: 1.5115e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_49_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_49_epoches.snn/assets


32932/32932 [==============================] - 1244s 38ms/step - loss: 1.4913e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_50_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_50_epoches.snn/assets


Time:  23098.130873547
[0.00021889498748350888, 0.0002177882706746459, 0.00020645371114369482, 0.00020231990492902696, 0.00019640893151517957, 0.0001888668048195541, 0.00018250700668431818, 0.00018129807722289115, 0.00017558435501996428, 0.00017260042659472674, 0.00016629110905341804, 0.00016865193902049214, 0.00015742740652058274, 0.00015942826576065272, 0.00015723073738627136, 0.00014870226732455194, 0.00015114957932382822, 0.0001491251605330035]


In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_50_epoches.snn')
start = timeit.default_timer()
for i in range(50, 75):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1308s 40ms/step - loss: 1.3904e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_51_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_51_epoches.snn/assets


32932/32932 [==============================] - 1295s 39ms/step - loss: 1.4183e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_52_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_52_epoches.snn/assets


32932/32932 [==============================] - 1289s 39ms/step - loss: 1.3911e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_53_epoches.snn/assets


...

32932/32932 [==============================] - 1269s 39ms/step - loss: 9.8259e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_74_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_74_epoches.snn/assets


32932/32932 [==============================] - 1294s 39ms/step - loss: 9.7767e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_75_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_75_epoches.snn/assets


Time:  32156.295902377
[0.0001390360703226179, 0.00014183383609633893, 0.0001391127152601257, 0.00013006503286305815, 0.00013723736628890038, 0.0001324775948887691, 0.00013205224240664393, 0.00012972537660971284, 0.00012532340770121664, 0.00012295950728002936, 0.00012172220885986462, 0.00011838213686132804, 0.00011702068150043488, 0.00011451079626567662, 0.00011292614362901077, 0.00011566513421712443, 0.00011173773236805573, 0.00011022460967069492, 0.00011024205741705373, 0.0001010570049402304, 0.00010674793156795204, 0.00010472891881363466, 0.00010046796523965895, 9.825887536862865e-05, 9.776655497262254e-05]


In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_75_epoches.snn')
start = timeit.default_timer()
for i in range(75, 100):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1294s 39ms/step - loss: 9.7511e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_76_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_76_epoches.snn/assets


32932/32932 [==============================] - 1300s 39ms/step - loss: 9.5788e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_77_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_77_epoches.snn/assets


32932/32932 [==============================] - 1255s 38ms/step - loss: 9.4515e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_78_epoches.snn/assets


...

32932/32932 [==============================] - 1260s 38ms/step - loss: 7.7819e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_99_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_99_epoches.snn/assets


32932/32932 [==============================] - 1296s 39ms/step - loss: 7.8164e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_100_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Merge_model_1.4.4.2/SNN_100_epoches.snn/assets


Time:  32016.878397557
[9.751132893143222e-05, 9.578766912454739e-05, 9.45151987252757e-05, 9.502568718744442e-05, 8.94939512363635e-05, 9.307992877438664e-05, 9.320741082774475e-05, 9.048885840456933e-05, 9.073789260582998e-05, 8.585557225160301e-05, 9.136200969805941e-05, 8.91434756340459e-05, 8.614898979431018e-05, 8.661900938022882e-05, 8.471386536257342e-05, 8.057040395215154e-05, 8.540929411537945e-05, 8.327089744852856e-05, 8.03721122792922e-05, 7.668460602872074e-05, 8.416632044827566e-05, 8.084389264695346e-05, 8.035721111809835e-05, 7.781856402289122e-05, 7.816377910785377e-05]


### Train Cosine

In [ ]:
losses = []
start = timeit.default_timer()
for i in range(20):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Cosine_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1297s 39ms/step - loss: 0.0373
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Cosine_model_1.4.4.2/SNN_1_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Cosine_model_1.4.4.2/SNN_1_epoches.snn/assets


32932/32932 [==============================] - 1293s 39ms/step - loss: 0.0149
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Cosine_model_1.4.4.2/SNN_2_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Cosine_model_1.4.4.2/SNN_2_epoches.snn/assets


32932/32932 [==============================] - 1287s 39ms/step - loss: 0.0125
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Cosine_model_1.4.4.2/SNN_3_epoches.snn/assets


...

32932/32932 [==============================] - 1247s 38ms/step - loss: 0.0064
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Cosine_model_1.4.4.2/SNN_19_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Cosine_model_1.4.4.2/SNN_19_epoches.snn/assets


32932/32932 [==============================] - 1221s 37ms/step - loss: 0.0063
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Cosine_model_1.4.4.2/SNN_20_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/Cosine_model_1.4.4.2/SNN_20_epoches.snn/assets


Time:  24599.488757659
[0.024486105889081955, 0.014937026426196098, 0.01245044730603695, 0.011058277450501919, 0.010084070265293121, 0.00939374603331089, 0.008840606547892094, 0.008458749391138554, 0.008067813701927662, 0.007809730246663094, 0.007627024780958891, 0.007439479697495699, 0.007421344518661499, 0.007130536716431379, 0.006980838254094124, 0.006809983868151903, 0.006638312246650457, 0.006449973676353693, 0.00643547996878624, 0.006330437492579222]
